In [1]:
import json
import os
import time
import ipyparallel as ipp

import matplotlib.pyplot as plt
import torch
import dfca as DFCA
import ifca as IFCA

import numpy as np

from util import *

Reads Config file and prepares the arguments you can choose in the config_dfca.json/config_ifca.json

In [2]:
rc = ipp.Cluster(n=3).start_and_connect_sync()
rc.activate()

rc.ids 

dview = rc[:]

Starting 3 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/3 [00:00<?, ?engine/s]

In [3]:
p_rates = [0.1, 0.2, 0.3, 0.4, 0.5]

dfca_gi = []
ifca = []
dfca_li = []

In [4]:
%%px --targets all

import sys
import os
import random
sys.path.append(os.getcwd())

random.seed(42)
seeds = random.sample(range(10000), 5)
p_rates = [0.1, 0.2, 0.3, 0.4, 0.5]

dfca_gi = []
ifca = []
dfca_li = []

Running the dfca-gi experiment

In [5]:
%%px --target 0 --noblock
import dfca as DFCA
import json
import time
import torch
import numpy as np


with open("config.json", "r") as read_file:
    config = json.load(read_file)

for pr in p_rates:
    config['participation_rate'] = pr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------DFCA-GI, p_rate: {pr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainCIFARCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_gi.append(res_mean)



<AsyncResult(%px): pending>

Running the dfca-li experiments

In [6]:
%%px --target 1 --noblock
import dfca as DFCA
import json
import time
import torch
import numpy as np


with open("config_li.json", "r") as read_file:
    config = json.load(read_file)

for pr in p_rates:
    config['participation_rate'] = pr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------DFCA-LI, p_rate: {pr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainCIFARCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_li.append(res_mean)



<AsyncResult(%px): pending>

Running the ifca experiment

In [7]:
%%px --target 2 --noblock
import ifca as IFCA
import json
import time
import torch
import numpy as np

with open("config.json", "r") as read_file:
    config = json.load(read_file)

for pr in p_rates:
    config['participation_rate'] = pr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------IFCA, p_rate: {pr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = IFCA.TrainCIFARCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    ifca.append(res_mean)



<AsyncResult(%px): pending>

## Results

DFCA-GI Experiment Outputs

In [8]:
rc.wait_interactive()
_.display_outputs()

dfca_gi = rc[0].pull('dfca_gi', block=True)
dfca_li = rc[1].pull('dfca_li', block=True)
dfca_li = rc[2].pull('ifca', block=True)

unknown:   0%|          | 0/3 [00:00<?, ?tasks/s]

---------------------------------IFCA, p_rate: 0.1, seed: 1824---------------------------------
config: {'m': 200, 'm_test': 40, 'p': 2, 'n': 500, 'uneven': False, 'local_model_init': False, 'participation_rate': 0.1, 'h1': 128, 'num_epochs': 30, 'batch_size': 50, 'tau': 5, 'lr': 0.25, 'data_seed': 1824, 'train_seed': 1824, 'config_override': '', 'project_dir': 'output_dfca_gi'}
Using device: cuda:3
num_data: 50000
num_data: 10000
finding good initializer from train data
Epoch -1 tr: l 2.303 a 0.105 clct[np.int64(7), np.int64(13)] clct_ans[np.int64(9), np.int64(11)]  5.457sec
found good initializer
Epoch -1 tr: l 2.302 a 0.110 clct[np.int64(11), np.int64(9)] clct_ans[np.int64(9), np.int64(11)]  4.307sec
Epoch -1 tst: l 2.303 a 0.103 clct[np.int64(21), np.int64(19)] clct_ans[np.int64(20), np.int64(20)]  3.348sec
Epoch 0 tr: l 2.212 a 0.224 clct[np.int64(7), np.int64(13)] clct_ans[np.int64(12), np.int64(8)]  lr 0.250000 13.419sec(train) 4.360sec(infer)
Epoch 0 tst: l 2.214 a 0.220 clct[n

100%|██████████| 170M/170M [00:07<00:00, 23.0MB/s]


IFCA Experiment Outputs

In [9]:
__.display_outputs()

---------------------------------DFCA-LI, p_rate: 0.1, seed: 1824---------------------------------
config: {'m': 200, 'm_test': 40, 'p': 2, 'n': 500, 'uneven': False, 'local_model_init': True, 'participation_rate': 0.1, 'h1': 128, 'num_epochs': 30, 'batch_size': 50, 'tau': 5, 'lr': 0.25, 'data_seed': 1824, 'train_seed': 1824, 'config_override': '', 'project_dir': 'output_dfca_li'}
Using device: cuda:3
num_data: 50000
num_data: 10000
finding good initializer from train data
Epoch -1 tr: l 2.302 a 0.101 clct[np.int64(6), np.int64(14)] clct_ans[np.int64(9), np.int64(11)] cl_acc0.650  5.770sec
Epoch -1 tr: l 2.302 a 0.102 clct[np.int64(12), np.int64(8)] clct_ans[np.int64(9), np.int64(11)] cl_acc0.550  4.404sec
found good initializer
Epoch -1 tr: l 2.303 a 0.104 clct[np.int64(13), np.int64(7)] clct_ans[np.int64(9), np.int64(11)] cl_acc0.500  4.332sec
Epoch -1 tst: l 2.302 a 0.110 clct[np.int64(19), np.int64(21)] clct_ans[np.int64(20), np.int64(20)] cl_acc0.525  9.997sec
Epoch 0 tr: l 2.303 

100%|██████████| 170M/170M [00:07<00:00, 23.2MB/s]


In [10]:
___.display_outputs()

---------------------------------DFCA-GI, p_rate: 0.1, seed: 1824---------------------------------
config: {'m': 200, 'm_test': 40, 'p': 2, 'n': 500, 'uneven': False, 'local_model_init': False, 'participation_rate': 0.1, 'h1': 128, 'num_epochs': 30, 'batch_size': 50, 'tau': 5, 'lr': 0.25, 'data_seed': 1824, 'train_seed': 1824, 'config_override': '', 'project_dir': 'output_dfca_gi'}
Using device: cuda:3
num_data: 50000
num_data: 10000
finding good initializer from train data
Epoch -1 tr: l 2.302 a 0.103 clct[np.int64(13), np.int64(7)] clct_ans[np.int64(9), np.int64(11)] cl_acc0.500  4.535sec
found good initializer
Epoch -1 tr: l 2.302 a 0.103 clct[np.int64(8), np.int64(12)] clct_ans[np.int64(9), np.int64(11)] cl_acc0.550  4.271sec
Epoch -1 tst: l 2.302 a 0.110 clct[np.int64(20), np.int64(20)] clct_ans[np.int64(20), np.int64(20)] cl_acc0.550  9.722sec
Epoch 0 tr: l 2.294 a 0.112 clct[np.int64(16), np.int64(4)] clct_ans[np.int64(12), np.int64(8)] cl_acc0.500  lr 0.250000 10.937sec(train) 

100%|██████████| 170M/170M [00:07<00:00, 23.6MB/s]


## Plots

In [11]:

r_ifca = {}
r_dfca_gi = {}
r_dfca_li = {}

r_ifca['test_loss'] = []
r_ifca['test_acc'] = []
r_ifca['cl_acc'] = []
r_ifca['train_loss'] = []
for i in range(len(p_rates)):
    r_ifca['test_loss'].append(ifca[i][0])
    r_ifca['test_acc'].append(ifca[i][1])
    r_ifca['cl_acc'].append(ifca[i][2])
    r_ifca['train_loss'].append(ifca[i][3])

r_dfca_li['test_loss'] = []
r_dfca_li['test_acc'] = []
r_dfca_li['cl_acc'] = []
r_dfca_li['train_loss'] = []
for i in range(len(p_rates)):
    r_dfca_li['test_loss'].append(dfca_li[i][0])
    r_dfca_li['test_acc'].append(dfca_li[i][1])
    r_dfca_li['cl_acc'].append(dfca_li[i][2])
    r_dfca_li['train_loss'].append(dfca_li[i][3])

r_dfca_gi['test_loss'] = []
r_dfca_gi['test_acc'] = []
r_dfca_gi['cl_acc'] = []
r_dfca_gi['train_loss'] = []
for i in range(len(p_rates)):
    r_dfca_gi['test_loss'].append(dfca_gi[i][0])
    r_dfca_gi['test_acc'].append(dfca_gi[i][1])
    r_dfca_gi['cl_acc'].append(dfca_gi[i][2])
    r_dfca_gi['train_loss'].append(dfca_gi[i][3])


os.makedirs('graphs', exist_ok=True)

for l in range(len(p_rates)):

    # plt.figure(figsize=(10,5))
    # plt.plot(r_ifca['train_loss'][l], label='ifca', color='xkcd:azure')
    # plt.plot(r_dfca_li['train_loss'][l], label='dfca-li', color='red')
    # plt.plot(r_dfca_gi['train_loss'][l], label='dfca-gi', color='green')
    # plt.xlabel('epoch', fontsize=20)
    # plt.ylabel('loss', fontsize=20)
    # plt.title(f'Train Loss per Epoch (p_rate={p_rates[l]})', fontsize=22, fontweight='bold')
    # plt.tick_params(axis='both', which='major', labelsize=18)
    # plt.legend(fontsize=20)
    # plt.savefig(os.path.join('graphs', 'train_loss.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['test_loss'][l], label='ifca', color='xkcd:azure')
    plt.plot(r_dfca_li['test_loss'][l], label='dfca-li', color='red')
    plt.plot(r_dfca_gi['test_loss'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch', fontsize=20)
    plt.ylabel('loss', fontsize=20)
    plt.title(f'Test Loss per Epoch (p_rate={p_rates[l]})', fontsize=22, fontweight='bold')
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.legend(fontsize=20)
    plt.savefig(os.path.join('graphs', 'test_loss.png'))

    # plt.figure(figsize=(10,5))
    # plt.plot(r_ifca['test_acc'][l], label='ifca', color='xkcd:azure')
    # plt.plot(r_dfca_li['test_acc'][l], label='dfca-li', color='red')
    # plt.plot(r_dfca_gi['test_acc'][l], label='dfca-gi', color='green')
    # plt.xlabel('epoch', fontsize=20)
    # plt.ylabel('test accuracy', fontsize=20)
    # plt.title(f'Test Accuracy per Epoch (p_rate={p_rates[l]})', fontsize=22, fontweight='bold')
    # plt.tick_params(axis='both', which='major', labelsize=18)
    # plt.legend(fontsize=20)
    # plt.savefig(os.path.join('graphs', 'test_acc.png'))

    plt.figure(figsize=(10,7))
    plt.ylim([0, 1.1])
    plt.plot(r_ifca['cl_acc'][l][:31], label='ifca', color='xkcd:azure', marker='o')
    plt.plot(r_dfca_li['cl_acc'][l][:31], label='dfca-li', color='red', marker='o')
    plt.plot(r_dfca_gi['cl_acc'][l][:31], label='dfca-gi', color='green', marker='o')
    plt.xlabel('epoch', fontsize=20)
    plt.ylabel('cluster acc', fontsize=20)
    plt.legend(fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.savefig(os.path.join('graphs', 'cluster_acc.png'))

IndexError: list index out of range